<a href="https://colab.research.google.com/github/SorrowAddict/SK-FLY-AI/blob/main/240130_LangChain_05_Text_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai langchain tiktoken pypdf

In [2]:
import os

os.environ['OPENAI_API_KEY'] = '3a9d04966a3c43ca92de7be4b803d6f7'
os.environ['AZURE_OPENAI_ENDPOINT'] = 'https://skyfly-001.openai.azure.com/'
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-05-15'

In [3]:
from langchain.embeddings import AzureOpenAIEmbeddings
embeddings_model = AzureOpenAIEmbeddings(
                        model='dev-text-embedding-ada-002'
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.azure_openai.AzureOpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(


In [4]:
embeddings = embeddings_model.embed_documents(
                {
                    '안녕하세요',
                    '제 이름은 홍길동 입니다.',
                    '이름이 무엇인가요?',
                    '랭체인은 유용합니다.',
                    'Hello World'
                }
              )
len(embeddings), len(embeddings[0])

(5, 1536)

In [5]:
embedded_query_q = embeddings_model.embed_query('이 대화에 언급된 이름은 무엇입니까?')
embedded_query_a = embeddings_model.embed_query('이 대화에 언급된 이름은 홍길동 입니다.')
print(len(embedded_query_q),len(embedded_query_a))

1536 1536


In [6]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cos_sim(A,B):
  return dot(A,B)/(norm(A)*norm(B))

In [7]:
print(cos_sim(embedded_query_q, embedded_query_a))
print(cos_sim(embedded_query_q, embeddings[1]))
print(cos_sim(embedded_query_q, embeddings[2]))

0.9085512700834164
0.9216955277777744
0.8496821436212559


# Huggingface Embedding

In [8]:
!pip install sentence_transformers

## model들 뭉쳐있는 github 같은 꿀 같은 곳 !!
### https://huggingface.co/

In [9]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = 'BAAI/bge-small-en'                # https://huggingface.co/BAAI/bge-small-en 에서 가져다 쓸 모델 이름
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings':True}

hf = HuggingFaceBgeEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
embeddings = hf.embed_documents(
              [
                  'today is monday',
                  'weather is nice today',
                  "what's the problem",
                  'langchain is useful',
                  'Hello World',
                  'my name is morris'
              ]
)

In [11]:
BGE_query_q = hf.embed_query('Hello? who is this?')
BGE_query_a = hf.embed_query('hi this is harrison')

print(cos_sim(BGE_query_q, BGE_query_a))
print(cos_sim(BGE_query_q, embeddings[1]))
print(cos_sim(BGE_query_q, embeddings[3]))

0.8522539947438873
0.7469068511786857
0.7576970760229004
